## Backpropagation.

Efficiently evaluate the gradient of a loss function with respect to the weights of a neural network.  Enables iterative tuning of weights of NN to minimize loss function and improve accuracy of the NN.